# Creating Hec-Ras plan and Unsteady flow files
This script was initially designed to create the Plan and Unsteady flow files required as an input to Hec-Ras. It did this by using a template of the structure of each of these files, and filling in the relevant scenario name and precipitation values. 

However, this didn't work as even those these text files appeared to be identical to those produced by Hec-Ras they were not being recognised. I think this is something to do with the fact that when you produce the files in Hec-Ras it stores some internal reference to the name of the scenario, which if doesn't exist then it doesn't think to look for the files (although, not sure if this is correct or not).  

So, instead reverted to creating unsteady flow and plan files from within Hec-Ras. This script does contain a code cell for reading the unsteady flow files produced in Hec-Ras and checking them against the precipitation data read in from file to ensure it is the correct data.

#### Code initialisation

In [7]:
import pandas as pd 
from math import log10, floor
import re
import os

model_directory ='../../../FloodModelling/MeganModel_v3 - Copy'

#### Functions for producing the files

In [2]:
def round_sig(x, sig=7):
    if x == 0:
        return 0
    elif x <0.1:
        return round(x,sig)
    else: 
        return round(x, sig-int(floor(log10(abs(x))))-1)

def format_number(x):
    if x == 0:
        return '0       '
    else:
        str_x = format(x, '.7f')
        str_x = str_x.lstrip('0')
        if x>1:
            str_x = ' ' + str_x
        return str_x

def create_unsteady_flow_file (template_fp, precip_fp, output_fp, flow_title):
    
    with open(template_fp) as f:
        contents = f.read()

    ### Read in precipitation values which we want to insert into the string
    precip = pd.read_csv(precip_fp)
    precip = precip['Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model']
    precip= precip[0:360]
    precip = precip.apply(round_sig)
    precip = precip.apply(format_number)
    precip = precip.str.cat(sep='')
    precip = re.sub("(.{80})", "\\1\n", precip, 0, re.DOTALL)

    ### Add the new precipitation values to the whole string
    contents = contents.replace('precip_values', precip)
    
    # Replace flow title
    if '6h' not in flow_title:
        flow_title = '6h_Cluster{}'.format(flow_title)
    contents = contents.replace("flow_title", flow_title)
    
    ### Save to text file
    with open(output_fp, 'w') as f:
        f.write(contents)

def create_plan (plan_title, short_identifier, unsteadyflow_file, output_fp):
    # Read file
    with open(model_directory + 'DissModel.p01') as f:
        contents = f.read()
    if '6h' not in plan_title:
        plan_title = '6h_Cluster{}'.format(plan_title)

    # Replace the variable with the correct values
    contents = contents.replace("6h_single-peak", plan_title)
    contents = contents.replace("6h_sp", short_identifier)
    contents = contents.replace("u01", unsteadyflow_file.split('DissModel.')[1])
    
    ### Save to text file
    with open(output_fp, 'w') as f:
        f.write(contents)

#### Code to produce the unsteady flow and plan files 
Now redundant as the files produced aren't recognised by Hec-Ras

In [23]:
# short_ids = ['6h_sp', '6h_dt', '6h_spt', '6h_ms', '6h_c1','6h_c2','6h_c3','6h_c4', '6h_c5', '6h_c6','6h_c7',
#              '6h_c8','6h_c9','6h_c10', '6h_c11', '6h_c12','6h_c13','6h_c14','6h_c15',]   
# methods = ['6h_single-peak', '6h_divide-time', '6h_subpeak-timing', '6h_max-spread', '1', '2', '3', '4', '5',
#            '6', '7', '8', '9','10', '11', '12', '13', '14', '15']  

# template_fp =  model_directory + '/template_unsteadyflowfile.txt'

# for method_number, method in enumerate(methods, start=1):
#     # Read the precipitation data for this method
#     print(method)
#     if '6h' in method:
#         precip_file = "../CreateSyntheticRainfallEvents/MultiplePeaks/PostLossRemoval/6h/{}_urban.csv".format(method)
#     else:
#         precip_file = "../CreateSyntheticRainfallEvents/RobertoProfiles/PostLossRemoval/6hr_100yrRP/cluster{}_urban_summer.csv".format(method)
    
#     # Define filepaths to save the unsteady flow file and plan file to
#     if method_number <10:
#         unsteadyflow_output_fp = model_directory + '/DissModel.u0{}'.format(method_number)
#         plan_output_fp =model_directory + 'DissModel.p0{}'.format(method_number)
#     else:
#         unsteadyflow_output_fp = model_directory + 'DissModel.u{}'.format(method_number)
#         plan_output_fp = model_directory + 'DissModel.p{}'.format(method_number)
        
#     # Define the unsteady flow file number (used as an input to the plan)
#     unsteadyflow_file_number = 'u0{}'.format(method_number)
#     if method != '6h_single-peak':
#         # Create unsteady flow and then create the plan
#         create_unsteady_flow_file(template_fp, precip_file, unsteadyflow_output_fp, method)
# #         create_plan(method, short_ids[method_number-1], unsteadyflow_output_fp, plan_output_fp)


## Cross checking unsteady flow files against precipitation data

In [353]:
for method_number in range(1,20):
    # Zero pad single digit numbers
    if len(str(method_number))==1:
        method_number = str(method_number).zfill(2)
    
    # Read in the unsteady flow file being used in Hec-Ras
    with open(model_directory + '/DissModel.u{}'.format(method_number)) as f:
        lines = f.read()
        
    # Extract the precipitation values to a list
    # Slight variations in formatting which have to be dealt with
    start=lines.find('Precipitation Hydrograph= 360')+len('Precipitation Hydrograph= 360')     
    end=lines.find('DSS Path') 
    hecras_precip_values = lines[start:end]
    hecras_precip_values = hecras_precip_values.replace('\n', ' ')
    hecras_precip_values = hecras_precip_values.replace('  ', '', 1)
    if hecras_precip_values.startswith(' '):
        hecras_precip_values = hecras_precip_values.replace(' ', '', 1)
    hecras_precip_values = wrap(hecras_precip_values, 8)
    hecras_precip_values
    
    # Read in precipitation value from file
    flow_title = lines[lines.find('6h_')+len('6h_'):lines.find('\n')].replace('C', 'c')
    if method_number in ['01','02','03','04']:
        precip_data =pd.read_csv("../CreateSyntheticRainfallEvents/MultiplePeaks/PostLossRemoval/6h/6h_{}_urban.csv".format(flow_title)) 
    else:
        precip_data =pd.read_csv("../CreateSyntheticRainfallEvents/RobertoProfiles/PostLossRemoval/6hr_100yrRP/{}_urban_summer.csv".format(flow_title))
    # Select the relevant column and round the data
    precip_data = round(precip_data['Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model'][:360],5)

    # Check if each value is the same
    # realFalses allows for slight differences in rounding
    Falses = []
    realFalses = []
    for i in range(0,360):
        if precip_data[i] != round(float(hecras_precip_values[i]),5):
            Falses.append(i)
            if abs(precip_data[i] - round(float(hecras_precip_values[i]),5)) > 0.001:
                realFalses.append(i)
    print(method_number, flow_title, len(Falses), len(realFalses))

01 single-peak 2 0
02 divide-time 0 0
03 subpeak-timing 1 0
04 max-spread 0 0
05 cluster1 2 0
06 cluster2 2 0
07 cluster3 4 0
08 cluster4 2 0
09 cluster5 2 0
10 cluster6 0 0
11 cluster7 0 0
12 cluster8 2 0
13 cluster9 1 0
14 cluster10 2 0
15 cluster11 1 0
16 cluster12 2 0
17 cluster13 0 0
18 cluster14 1 0
19 cluster15 1 0
